In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import xarray as xr
from datetime import datetime
import cv2
import tabular.subset_by_images
import matplotlib.pyplot as plt
import cv_experiments.cv_common as cc
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
image_to_boat = lambda fname: os.path.basename(fname).split('_')[0].split('-')[1]
image_to_datetime = lambda fname: np.datetime64(tabular.subset_by_images.image_to_datetime(fname))
image_to_camera = lambda fname: os.path.splitext(os.path.basename(fname))[0].split('_')[2].split('-')[1]

In [ ]:
def pr2line(pr, shape):
    pitch_slope,roll_slope = pr
    height,width = shape[:2]
    y0 = height*(0.5-pitch_slope)
    yoffset = -roll_slope*width/2
    return (0, int(y0-yoffset)), (width, int(y0+yoffset))

def show_images(images):
    for row in images:
        lat, lon = None, None
        if "dataframe" in row:
            lat, lon = row["dataframe"][["latitude", "longitude"]].mean()
        print(f"{row['boat']} at {pd.to_datetime(row['datetime']):%Y-%m-%d %H:%M:%S}, {f'lat={lat:.3f}, lon={lon:.3f}, ' if lat and lon else ''}camera {row['camera']}:")
        if "msg" in row: print(row["msg"])
        img = cv2.cvtColor(row["mat"], cv2.COLOR_GRAY2RGB)
        if "horizon_inferred" in row:
            c1,c2 = pr2line(row["horizon_inferred"], img.shape)
            cv2.line(img, c1, c2, (0, 0, 255), 3, cv2.LINE_AA)
        if "horizon_manual" in row:
            c1,c2 = pr2line(row["horizon_manual"], img.shape)
            cv2.line(img, c1, c2, (255, 0, 0), 3, cv2.LINE_AA)
        plt.imshow(img, vmin=0, vmax=255)
        plt.show()

In [ ]:
horizon_table = pd.read_csv("../tabular/horizons/horizons.csv")
images = []
for _,row in horizon_table.iterrows():
    path = os.path.join("../to_be_horizoned", row["filename"])
    this_img = {
        "boat": image_to_boat(path),
        "datetime": image_to_datetime(path),
        "camera": image_to_camera(path),
        "mat": cc.get_image(path),
        "horizon_manual": (row["pitchFrac"], row["rollSlope"])
        }
    images.append(this_img)
show_images(images[:1])  # Sanity check

In [ ]:
excerpt_to_boat = lambda fname: os.path.splitext(os.path.basename(fname))[0].split('_')[1]
excerpt_paths = sorted(f.path for f in os.scandir("../tabular/excerpts") if f.is_file() and os.path.basename(f.path)[0] != '.')
excerpts = {excerpt_to_boat(fname): xr.load_dataset(fname) for fname in excerpt_paths}

In [ ]:
wide_time = np.timedelta64(10, "s")
relevant_variables = ["ROLL", "PITCH", "HDG", "ROLL_WING", "PITCH_WING", "HDG_WING", "WING_ANGLE", "latitude", "longitude"]

for row in images:
    timeslice = slice(row["datetime"]-wide_time, row["datetime"]+wide_time)
    row["dataframe"] = excerpts[row["boat"]].sel(time=timeslice).to_dataframe()[relevant_variables]
show_images(images[:1])

In [ ]:
# Let's make some plots!
# x-axes are wing pitch, wing roll, etc.
# y-axes are observed roll
# One series per (boat, camera) combination

narrow_time_before = np.timedelta64(int(0.0*1000), "ms")
narrow_time_after = np.timedelta64(int(0.25*1000), "ms")
combos = [(b, c) for b in set(row["boat"] for row in images) for c in set(row["camera"] for row in images)]
print(combos)
def do_plot(x_var, y_var):
    yi = {"pitchFrac": 0, "rollSlope": 1}[y_var]
    for k in combos:
        boat, camera = k
        relevant = filter(lambda row: row["boat"] == boat and row["camera"] == camera, images)
        x,y = zip(*[(row["dataframe"][x_var][slice(row["datetime"]-narrow_time_before, row["datetime"]+narrow_time_after)].mean(), row["horizon_manual"][yi]) for row in relevant])
        plt.scatter(x, y, s=3, label=k)
        plt.xlabel(x_var)
        plt.ylabel(y_var)
    plt.legend()
    plt.show()
do_plot("PITCH_WING", "rollSlope")
do_plot("ROLL_WING", "rollSlope")
do_plot("PITCH_WING", "pitchFrac")
do_plot("ROLL_WING", "pitchFrac")
do_plot("WING_ANGLE", "rollSlope")
do_plot("WING_ANGLE", "pitchFrac")